In [1]:
import joblib
import json
import pandas
import warnings
import numpy

import sklearn.neural_network
import sklearn.model_selection
import sklearn.exceptions

warnings.filterwarnings("ignore", category=sklearn.exceptions.ConvergenceWarning)

In [2]:
data_train = pandas.read_csv('../data/features.train.csv').set_index('id')
data_train

,fnlwgt,age-group,education-num,capitalgain,capitalloss,hoursperweek,workclass_self-emp-not-inc,workclass_private,workclass_self-emp-inc,workclass_local-gov,...,native-country_honduras,native-country_thailand,native-country_laos,native-country_peru,native-country_holand-netherlands,native-country_haiti,native-country_scotland,sex_male,sex_female,label
id,,,,,,,,,,,,,,,,,,,,,
2103,1.395470,2.0,14.0,0.0,4.0,2.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
14649,-0.873898,0.0,8.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
7379,-0.592186,2.0,8.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
24479,-1.349595,2.0,9.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
19532,2.041210,3.0,1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8695,-1.405521,0.0,9.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2192,-0.821560,2.0,9.0,2.0,0.0,2.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
8250,1.210247,0.0,8.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [3]:
model = sklearn.neural_network.MLPClassifier(
    hidden_layer_sizes=(20, 10),
    solver='sgd', 
    activation='logistic',
    alpha=0.0,  
    batch_size=32,
    learning_rate='constant',
    learning_rate_init=0.1,
    max_iter=1000,
    momentum=0.0, 
    nesterovs_momentum=False, 
    validation_fraction=0.0,
    shuffle=True,
    random_state=0
)

In [4]:
search_parameters = {
    'alpha': [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1]
}

In [5]:
optimizer = sklearn.model_selection.GridSearchCV(
    estimator=model, 
    param_grid=search_parameters,
    n_jobs=-1,
    verbose=2
)

In [6]:
optimizer.fit(
    data_train.drop(['label'], axis='columns'),
    data_train['label'],
)

Fitting 5 folds for each of 19 candidates, totalling 95 fits


GridSearchCV(estimator=MLPClassifier(activation='logistic', alpha=0.0,
                                     batch_size=32, hidden_layer_sizes=(20, 10),
                                     learning_rate_init=0.1, max_iter=1000,
                                     momentum=0.0, nesterovs_momentum=False,
                                     random_state=0, solver='sgd',
                                     validation_fraction=0.0),
             n_jobs=-1,
             param_grid={'alpha': [0.001, 0.002, 0.003, 0.004, 0.005, 0.006,
                                   0.007, 0.008, 0.009, 0.01, 0.02, 0.03, 0.04,
                                   0.05, 0.06, 0.07, 0.08, 0.09, 0.1]},
             verbose=2)

In [7]:
cv_results = pandas.DataFrame(optimizer.cv_results_)
cv_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,57.776910,8.722605,0.010961,0.000279,0.001,{'alpha': 0.001},0.817225,0.824242,0.827432,0.822010,0.819139,0.822010,0.003620,2
1,46.543749,10.087567,0.011510,0.001075,0.002,{'alpha': 0.002},0.818182,0.829027,0.827113,0.822329,0.818820,0.823094,0.004344,1
2,35.672179,8.112353,0.010845,0.000134,0.003,{'alpha': 0.003},0.819139,0.819777,0.809250,0.816587,0.818820,0.816715,0.003884,3
3,34.834363,8.560083,0.010727,0.000159,0.004,{'alpha': 0.004},0.813716,0.821372,0.808293,0.816268,0.817225,0.815375,0.004315,4
4,35.308313,7.702056,0.012057,0.002730,0.005,{'alpha': 0.005},0.813716,0.821691,0.808612,0.815311,0.815311,0.814928,0.004180,7
5,30.114984,11.683473,0.010571,0.000038,0.006,{'alpha': 0.006},0.809888,0.821691,0.807337,0.814673,0.816268,0.813971,0.005021,10
6,30.087252,12.108601,0.010632,0.000299,0.007,{'alpha': 0.007},0.809888,0.822329,0.807337,0.813397,0.817225,0.814035,0.005318,9
7,30.402444,12.496725,0.010343,0.000133,0.008,{'alpha': 0.008},0.809569,0.821372,0.806380,0.812759,0.817544,0.813525,0.005385,12
8,30.728289,12.505539,0.010421,0.000150,0.009,{'alpha': 0.009},0.809888,0.821372,0.805742,0.811164,0.817863,0.813206,0.005644,13
9,26.162510,9.247835,0.010386,0.000134,0.01,{'alpha': 0.01},0.809569,0.821053,0.804785,0.821372,0.815630,0.814482,0.006483,8


In [8]:
cv_results.sort_values(['rank_test_score'])

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
1,46.543749,10.087567,0.011510,0.001075,0.002,{'alpha': 0.002},0.818182,0.829027,0.827113,0.822329,0.818820,0.823094,0.004344,1
0,57.776910,8.722605,0.010961,0.000279,0.001,{'alpha': 0.001},0.817225,0.824242,0.827432,0.822010,0.819139,0.822010,0.003620,2
2,35.672179,8.112353,0.010845,0.000134,0.003,{'alpha': 0.003},0.819139,0.819777,0.809250,0.816587,0.818820,0.816715,0.003884,3
3,34.834363,8.560083,0.010727,0.000159,0.004,{'alpha': 0.004},0.813716,0.821372,0.808293,0.816268,0.817225,0.815375,0.004315,4
10,21.855759,4.899202,0.010389,0.000250,0.02,{'alpha': 0.02},0.810845,0.826794,0.800638,0.820734,0.817544,0.815311,0.008961,5
11,19.828504,4.150471,0.010268,0.000128,0.03,{'alpha': 0.03},0.810845,0.829027,0.797767,0.820734,0.817544,0.815183,0.010490,6
4,35.308313,7.702056,0.012057,0.002730,0.005,{'alpha': 0.005},0.813716,0.821691,0.808612,0.815311,0.815311,0.814928,0.004180,7
9,26.162510,9.247835,0.010386,0.000134,0.01,{'alpha': 0.01},0.809569,0.821053,0.804785,0.821372,0.815630,0.814482,0.006483,8
6,30.087252,12.108601,0.010632,0.000299,0.007,{'alpha': 0.007},0.809888,0.822329,0.807337,0.813397,0.817225,0.814035,0.005318,9
5,30.114984,11.683473,0.010571,0.000038,0.006,{'alpha': 0.006},0.809888,0.821691,0.807337,0.814673,0.816268,0.813971,0.005021,10


In [9]:
optimizer.best_params_

{'alpha': 0.002}